# Type별로 키워드 전환

In [ ]:
import os, json
from tools import ONEDRIVE_PATH

i = 1
mode = '금융일반'

ONEDRIVE_NEW = os.path.join(f'{ONEDRIVE_PATH}'.replace('selectstar', '데이터검수'), '재설계', '1. Q_type_label', mode)
origin_type = json.load(open(os.path.join(ONEDRIVE_NEW, f'E{i}_{mode}_L2.json'), 'r'))

In [ ]:
# 분류 유형
# - a : 단일 개념에 대한 부정형
# - b : 단일 개념에 대한 긍정형
# - c : 박스형 / 조합형
# - d : 계산 및 수치 도출형
# - e : 용어 정의 및 지식 확인형
# - f : 빈칸 채우기 및 순서 나열형

types = {}
for q in origin_type:
    q_type = q.get('type', 'Unknown')
    if q_type in types:
        types[q_type].append(q)
    else:
        types[q_type] = [q]

# 결과 출력
print(f"총 문제 수: {len(origin_type)}")
print(f"\nType별 분류:")
for type_name, count in sorted(types.items()):
    print(f"  {type_name}: {len(count)}개")

In [ ]:
# 계산 및 수치 도출형 - 얼마인가? 계산하시오
# 용어 정의 및 지식 확인형 - 설명하시오
# 빈칸 채우기 및 순서 나열형 - 채워넣으시오, 나열하시오
type_cast = {
    "a": "단일 개념에 대한 부정형",
    "b": "단일 개념에 대한 긍정형",
    "c": "박스 및 조합형",
    "d": "계산 및 수치 도출형",
    "e": "용어 정의 및 지식 확인형",
    "f": "빈칸 채우기 및 순서 나열형"
}

In [ ]:
what_type = 'f'

file_name = f'검수/{mode}_exam{i}_{type_cast[what_type]}.json'

with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(types[what_type], f, ensure_ascii=False, indent=4)

In [ ]:
from tools.transformed import *
from tqdm import tqdm

llm, onedrive_path, log_func = _init_common()
questions = _load_questions(file_name, log_func, '1단계')

In [ ]:
for q in questions:
    print(q['question'])
    print("="*30)

In [ ]:
# 서술형 문제로 변환 # d: 계산형
# system_prompt = """당신은 25년 경력의 서술형 문제 전문가입니다. 아래 지시사항을 정확히 이해하고 수행하여 서술형 문제로 변환하시오.

# 지시사항:
# 1. 주어진 question에서 계산하려는 대상을 식별하라.
# 2. 계산하려는 대상을 바탕으로 계산 및 수치 도출형 문제로 변환하라.
# 3. 단서조항, 수식, 조건, 표 등은 모두 유지하라. 
# 예시:
#     - A회사의 매출액은 100억원, 변동비율은 40%, 고정비용은 30억원, 이자비용이 1억원이다. A회사의 매출액이 80억원이 되면 영업이익은 얼마인가? -> A회사의 매출액은 100억원, 변동비율은 40%, 고정비용은 30억원, 이자비용이 1억원이다. A회사의 매출액이 80억원이 되면 영업이익을 구하시오.
#     - 한국의 수출액은 3,000억 달러이고 수입액은 2,500억 달러라고 하자. 한국인은 해외 자산을 700억 달러 매입하였고, 외국인은 한국에서 거래되는 자산을 800억 달러 매입하였다. 이때 한국의 외환보유액은 어떻게 변하는가? -> 한국의 수출액은 3,000억 달러이고 수입액은 2,500억 달러라고 하자. 한국인은 해외 자산을 700억 달러 매입하였고, 외국인은 한국에서 거래되는 자산을 800억 달러 매입하였다. 이때 한국의 외환보유액을 구하시오.
#     - ~ 아래 전략형게임의 순수전략 내쉬균형은 몇 개인가? -> ~ 아래 전략형게임의 순수전략 내쉬균형을 구하시오.

# 출력 형식:
# - 서술형 문제: [계산하려는 대상]을/를 구하시오. (단서조항, 수식, 조건, 표 등 모두 유지)
# """
# 입력:
    # - question: {q['question']}

# e: 용어 정의 및 지식 확인형
# system_prompt = """당신은 25년 경력의 서술형 문제 전문가입니다. 아래 지시사항을 정확히 이해하고 수행하여 서술형 문제로 변환하시오.

# 지시사항:
# 1. 주어진 answer에서 설명하려는 개념을 식별하라.
# 2. 단서조항, 수식, 조건, 표 등은 모두 유지하라.

# 출력 형식:
#  - 서술형 문제: [설명하려는 개념]에 대해 설명하시오. (단서조항, 수식, 조건, 표 등 모두 유지)
# """ 
#     입력:
#     - options: {q['options']}
#     - answer: {q['answer']}


log_func("서술형 문제 변환 중...")
for q in tqdm(questions[:1], desc="서술형 문제 변환"):
    _clean_question_data(q)
    
    user_prompt = f"""
    입력:
    - options: {q['options']}
    - answer: {q['answer']}

    """
    response = llm.query_openrouter(system_prompt, user_prompt, model_name='google/gemini-2.5-flash')
    q['essay_question'] = response.replace('서술형 문제: ', '').replace("[", "").replace("]", "").replace("-", "").strip()

In [ ]:
with open(file_name.replace('.json', '_transformed.json'), 'w', encoding='utf-8') as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

## 검수용으로 변환

In [ ]:
import os, json

i = 2
with open(f'exam{i}_cal_transformed.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

In [ ]:
new_qs = []
for q in questions:
    new_q = {
        'file_id': q['file_id'],
        'title': q['title'],
        'chapter': q['chapter'],
        'tag': q['tag'],
        'domain': q['domain'],
        'subdomain': q['subdomain'],
        # 'classification_reason': q['classification_reason'],
        'is_calculation': q['is_calculation'],
        'question': q['essay_question'],
        'options': q['options'],
        'answer': q['answer'],
        'explanation': q['explanation'],
        'type': q['type']
    }
    new_qs.append(new_q)

with open(f'exam{i}_cal_transformed_for_check.json', 'w', encoding='utf-8') as f:
    json.dump(new_qs, f, ensure_ascii=False, indent=4)

In [ ]:
with open(f'exam{i}_cal.json', 'r', encoding='utf-8') as f:
    questions = json.load(f)

new_qs = []
for q in questions:
    new_q = {
        'file_id': q['file_id'],
        'title': q['title'],
        'chapter': q['chapter'],
        'tag': q['tag'],
        'domain': q['domain'],
        'subdomain': q['subdomain'],
        # 'classification_reason': q['classification_reason'],
        'is_calculation': q['is_calculation'],
        'question': q['question'],
        'options': q['options'],
        'answer': q['answer'],
        'explanation': q['explanation'],
        'type': q['type']
    }
    new_qs.append(new_q)

with open(f'exam{i}_cal.json', 'w', encoding='utf-8') as f:
    json.dump(new_qs, f, ensure_ascii=False, indent=4)